# AIOK DE-NAS for Computer Vision (CNN and ViT) Demo

DE-NAS is a multi-model, hardware-aware, train-free NAS to construct compact model architectures for target platform directly. DE-NAS includes CNN-based search space for CV domain and Transformer-based search space for CV/NLP/ASR domains, and leverages hardware-aware train-free scoring method to evaluate the performance of the candidate architecture without training.

This demo mainly introduces CV integration with DE-NAS to search lighter, faster, higher performance cnn-based and transformer-based ASR model in a training-free way.

# Content
* [DE-NAS on CV Domain](#DE-NAS-on-CV-Domain)
* [Getting Started](#Getting-Started)
    * [Environment Setup](#Environment-Setup)
    * [Enter Docker](#Enter-Docker)
    * [Configuration](#Configuration)
    * [Launch Search](#Launch-Search)
    * [Launch Training with Best Searched Model Structure](#Launch-Training-with-Best-Searched-Model-Structure)

# DE-NAS on CV Domain

## CV CNN Supernet and Search Space

For CNN models, the basic structure is constructure by Residual block or Bottleneck block, the Search Space for CNN domain as following:
``` yaml
SUPERNET structure:
    "SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)
SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,1,1)"
SEARCH SPACE:
    number of layer:[18, 35, 50, 101]
    Convolutional layer kernel size:[1x1, 3x3, 5x5, 7x7, 9x9, 11x11]
    Number of filters:[8,16,32,64,128]
```

## CV ViT Supernet and Search Space

For VIT models, the basic structure is the transformer based model which generated from the unified transformer based search space, the Search Space for VIT domain as following:

``` yaml
SUPERNET:
  MLP_RATIO: 4.0 # Linear layer ratio
  NUM_HEADS: 10 # num of attention heads
  EMBED_DIM: 640 # Q,K,V embedding dimision
  DEPTH: 16 # number of transformer layers
SEARCH_SPACE:
  MLP_RATIO:[3.0,3.5,4.0]
  NUM_HEADS:[3,4,5,6,7,8,9,10]
  DEPTH:[12,13,14,15,16]
  EMBED_DIM:[192,216,240,320,384,448,528,576,624]
```

# Getting Started

## Environment Setup

``` bash
git clone https://github.com/intel/e2eAIOK.git
cd e2eAIOK
git submodule update --init –recursive
python3 scripts/start_e2eaiok_docker.py -b pytorch112 -w ${host0} ${host1} ${host2} ${host3} --proxy ""
```

## Enter Docker
``` bash
sshpass -p docker ssh ${host0} -p 12347
```

## Configuration

* Create config file for CNN model search

``` yaml
model_type: cnn
search_engine: EvolutionarySearchEngine # Options: random search, Evolution algorithm, SigOpt search
batch_size: 64
random_max_epochs: 1000
max_epochs: 1
select_num: 50
population_num: 50 # population number size for EA search
crossover_num: 0
mutation_num: 50 # mutation number size for EA search
budget_num_layers: 18 # number of layer threshold
budget_model_size: 1000000 # model parameter size threshold
budget_flops: 10000000 # model FLOPs threshold
img_size: 32 # input image size
num_classes: 100
plainnet_struct: "SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,1,1)" # This is the supernet architechture
no_reslink: False
no_BN: False
use_se: False
seed: 0
expressivity_weight: 1 # weight for train free score of expressivity 
complexity_weight: 0 # weight for train free score of complexity
diversity_weight: 0 # weight for train free score of diversity score
saliency_weight: 1 # weight for train free score of salience score
latency_weight: 0 # weight for latency setup according to different platforms
```

* Create config file for VIT model search

``` yaml
model_type: transformer
search_engine: EvolutionarySearchEngine # Options: random search, Evolution algorithm, SigOpt search
batch_size: 64
random_max_epochs: 1000
max_epochs: 10
select_num: 50
population_num: 50
m_prob: 0.2
s_prob: 0.4
crossover_num: 25
mutation_num: 25
max_param_limits: 100
min_param_limits: 1
supernet_cfg: /home/vmagent/app/e2eaiok/conf/denas/cv/supernet_vit/supernet_large.conf
img_size: 224
patch_size: 16 # patch number for input image
drop_rate: 0.0 # dropout ratio
drop_path_rate: 0.1
max_relative_position: 14 #max distance in relative position embedding
gp: True
relative_position: True
change_qkv: True
abs_pos: True
seed: 0
expressivity_weight: 0 # weight for train free score of expressivity 
complexity_weight: 0 # weight for train free score of complexity
diversity_weight: 1 # weight for train free score of diversity score
saliency_weight: 1 # weight for train free score of salience score
latency_weight: 10000 # weight for latency setup according to different platforms
```

* Create config file for CNN model train

``` yaml
domain: cnn
train_epochs: 1
eval_epochs: 1
input_size: 32
best_model_structure: /home/vmagent/app/e2eaiok/e2eAIOK/DeNas/best_cnn_model_structure.txt
num_classes: 10
dist_backend: gloo
train_batch_size: 128
eval_batch_size: 128
data_path: ~/data/pytorch_cifar10
data_set: CIFAR10
output_dir: ./
num_workers: 10
pin_mem: True
eval_metric: "accuracy"
learning_rate: 0.001
momentum: 0.9
weight_decay: 0.01
optimizer: "SGD"
criterion: "CrossEntropyLoss"
lr_scheduler: "CosineAnnealingLR"
print_freq: 10
metric_threshold: 94
mode: "train"
```

* Create config file for VIT model train

``` yaml
domain: vit
train_epochs: 1
eval_epochs: 1
input_size: 32
best_model_structure: /home/vmagent/app/e2eaiok/e2eAIOK/DeNas/best_vit_model_structure.txt
num_classes: 10
dist_backend: gloo
train_batch_size: 128
eval_batch_size: 128
data_path: ~/data/pytorch_cifar10
data_set: CIFAR10
output_dir: ./
num_workers: 10
pin_mem: True
eval_metric: "accuracy"
learning_rate: 0.001
momentum: 0.9
weight_decay: 0.01
optimizer: "SGD"
criterion: "CrossEntropyLoss"
lr_scheduler: "CosineAnnealingLR"
print_freq: 10
mode: "train"
gp: True
change_qkv: True 
relative_position: True # whether to use relative position embedding
drop_path: 0.1 #Drop path rate
max_relative_position: 14 #max distance in relative position embedding
no_abs_pos: False
patch_size: 16 # patch size for input image
drop: 0.0
metric_threshold: 94 # early stop target accuracy
SUPERNET:
  MLP_RATIO: 4.0
  NUM_HEADS: 10
  EMBED_DIM: 640
  DEPTH: 16
```

## Launch Search

### Launch Search for CNN
The input is the configuration for CNN domain, edit the configuration file of `e2eaiok_denas_cnn.conf` for CNN model search, and then run below commond line

In [7]:
%%bash
cd /home/vmagent/app/e2eaiok/e2eAIOK/DeNas
python -u /home/vmagent/app/e2eaiok/e2eAIOK/DeNas/search.py --domain cnn --conf /home/vmagent/app/e2eaiok/conf/denas/cv/e2eaiok_denas_cnn.conf

paths: /home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas/asr/utils, /home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas/asr
['/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/opt/intel/oneapi/advisor/2022.1.0/pythonapi', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python39.zip', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/lib-dynload', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/site-packages', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/site-packages/warprnnt_pytorch-0.1-py3.9-linux-x86_64.egg', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/site-packages/e2eAIOK-0.2.4-py3.9.egg', '', '/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/hom

12/02/2022 01:27:34 - INFO - DENAS -   random 18/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,40,2,24,2)SuperResK3K3(40,64,2,32,1)SuperConvK1BNRELU(64,128,1,1) nas_score 301.6855163574219 params 98532
SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK1K5K1(64,128,2,24,3)SuperConvK1BNRELU(128,128,1,1)
12/02/2022 01:27:34 - INFO - DENAS -   random 19/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK1K5K1(64,128,2,24,3)SuperConvK1BNRELU(128,128,1,1) nas_score 1175.869384765625 params 201836
SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,88,1,1)
12/02/2022 01:27:34 - INFO - DENAS -   random 20/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,

SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK1K3K1(16,48,2,8,3)SuperResK3K3(48,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,1,1)
12/02/2022 01:27:37 - INFO - DENAS -   random 38/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK1K3K1(16,48,2,8,3)SuperResK3K3(48,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,1,1) nas_score 659.6685791015625 params 110076
SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK1K3K1(16,48,2,16,2)SuperResK3K3(48,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,1,1)
12/02/2022 01:27:37 - INFO - DENAS -   random 39/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK1K3K1(16,48,2,16,2)SuperResK3K3(48,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,1,1) nas_score 398.29888916015625 params 116988
SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,24,2,40,1)SuperResK3K3(24,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,128,

12/02/2022 01:27:40 - INFO - DENAS -   mutation 7/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,32,2,24,1)SuperResK5K5(32,48,2,48,2)SuperConvK1BNRELU(48,128,1,1) nas_score 115.31932067871094 params 257420
SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,48,2,32,2)SuperResK3K3(48,64,2,32,1)SuperConvK1BNRELU(64,152,1,1)
12/02/2022 01:27:40 - INFO - DENAS -   mutation 8/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,48,2,32,2)SuperResK3K3(48,64,2,32,1)SuperConvK1BNRELU(64,152,1,1) nas_score 271.59783935546875 params 123532
SuperConvK3BNRELU(3,16,1,1)SuperResK1K3K1(16,40,1,8,1)SuperResK3K3(40,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,24,2,48,2)SuperConvK1BNRELU(24,128,1,1)
12/02/2022 01:27:40 - INFO - DENAS -   mutation 9/50 structure SuperConvK3BNRELU(3,16,1,1)SuperResK1K3K1(16,40,1,8,1)SuperResK3K3(40,32,2,16,1)SuperResK

SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,40,1,1)
12/02/2022 01:27:43 - INFO - DENAS -   mutation 27/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,64,2,32,1)SuperConvK1BNRELU(64,40,1,1) nas_score 62.43788528442383 params 87964
SuperConvK3BNRELU(3,16,1,1)SuperResK3K3(16,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK5K5(64,24,2,48,1)SuperConvK1BNRELU(24,128,1,1)
12/02/2022 01:27:43 - INFO - DENAS -   mutation 28/50 structure SuperConvK3BNRELU(3,16,1,1)SuperResK3K3(16,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK5K5(64,24,2,48,1)SuperConvK1BNRELU(24,128,1,1) nas_score 728.176025390625 params 164396
SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,40,2,24,2)SuperResK3K3(40,64,2,32,1)SuperConvK1BNRELU(64,48,1,1)

12/02/2022 01:27:46 - INFO - DENAS -   mutation 46/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK1K3K1(64,128,2,40,2)SuperConvK1BNRELU(128,160,1,1) nas_score 271.61358642578125 params 183020
SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,48,2,16,3)SuperConvK1BNRELU(48,64,1,1)
12/02/2022 01:27:46 - INFO - DENAS -   mutation 47/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK3K3(64,48,2,16,3)SuperConvK1BNRELU(48,64,1,1) nas_score 210.453369140625 params 96956
SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(32,64,2,32,1)SuperResK1K7K1(64,40,2,24,2)SuperConvK1BNRELU(40,88,1,1)
12/02/2022 01:27:46 - INFO - DENAS -   mutation 48/50 structure SuperConvK3BNRELU(3,8,1,1)SuperResK3K3(8,16,1,8,1)SuperResK3K3(16,32,2,16,1)SuperResK3K3(3

### Launch Search for ViT
The input is the configuration for VIT domain, edit the configuration file of `e2eaiok_denas_vit.conf` for vit model search, and then run below commond line

In [12]:
%%bash
cd /home/vmagent/app/e2eaiok/e2eAIOK/DeNas
python -u /home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas/search.py --domain vit --conf /home/vmagent/app/e2eaiok/e2eAIOK/conf/denas/cv/e2eaiok_denas_vit.conf

paths: /home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas/asr/utils, /home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas/asr
['/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/opt/intel/oneapi/advisor/2022.1.0/pythonapi', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python39.zip', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/lib-dynload', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/site-packages', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/site-packages/warprnnt_pytorch-0.1-py3.9-linux-x86_64.egg', '/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/site-packages/e2eAIOK-0.2.4-py3.9.egg', '', '/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas', '/hom

12/02/2022 02:02:02 - INFO - DENAS -   random 30/50 structure (14, 4.0, 3.5, 3.0, 3.0, 3.5, 4.0, 3.5, 4.0, 4.0, 3.0, 3.5, 3.5, 3.0, 3.5, 9, 5, 6, 4, 5, 5, 6, 3, 4, 10, 3, 5, 3, 7, 240) nas_score 260.6387023925781 params 10.64161
12/02/2022 02:02:09 - INFO - DENAS -   random 31/50 structure (15, 3.5, 3.0, 3.5, 3.5, 3.0, 3.0, 3.5, 3.5, 4.0, 3.0, 3.5, 3.5, 4.0, 4.0, 3.0, 6, 10, 4, 5, 3, 10, 9, 6, 7, 6, 3, 4, 5, 3, 10, 384) nas_score 289.3059997558594 params 24.696586
12/02/2022 02:02:16 - INFO - DENAS -   random 32/50 structure (16, 4.0, 3.0, 3.5, 4.0, 3.5, 3.0, 4.0, 3.5, 3.0, 4.0, 3.5, 3.5, 4.0, 3.0, 3.0, 3.5, 9, 3, 3, 5, 6, 7, 7, 9, 6, 6, 3, 9, 5, 5, 5, 10, 320) nas_score 244.2968292236328 params 20.000586
12/02/2022 02:02:26 - INFO - DENAS -   random 33/50 structure (16, 3.0, 3.0, 3.0, 3.5, 3.5, 4.0, 3.5, 3.0, 4.0, 3.0, 3.0, 3.0, 3.5, 3.5, 3.5, 4.0, 4, 7, 7, 9, 4, 7, 3, 6, 9, 7, 6, 9, 10, 10, 5, 5, 576) nas_score 235.0663604736328 params 52.550986
12/02/2022 02:02:31 - INFO - DENAS -  

12/02/2022 02:06:00 - INFO - DENAS -   mutation 16/25 structure (13, 3.5, 4.0, 3.0, 3.0, 4.0, 4.0, 3.5, 3.0, 3.5, 3.5, 3.5, 4.0, 4.0, 5, 7, 3, 4, 7, 9, 5, 10, 4, 4, 3, 9, 5, 448) nas_score 369.3968505859375 params 27.87569
12/02/2022 02:06:06 - INFO - DENAS -   mutation 17/25 structure (12, 3.0, 3.5, 4.0, 3.0, 4.0, 3.5, 4.0, 3.0, 3.5, 3.0, 3.0, 4.0, 7, 3, 10, 7, 5, 7, 6, 5, 5, 3, 7, 3, 448) nas_score 416.1216735839844 params 25.051882
12/02/2022 02:06:12 - INFO - DENAS -   mutation 18/25 structure (12, 4.0, 3.5, 4.0, 3.0, 3.0, 4.0, 3.5, 3.0, 3.0, 4.0, 3.5, 3.0, 5, 5, 6, 3, 3, 3, 5, 9, 6, 7, 6, 6, 448) nas_score 309.3833923339844 params 24.592362
12/02/2022 02:06:17 - INFO - DENAS -   mutation 19/25 structure (12, 3.5, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 3.5, 4.0, 3.5, 3.5, 3, 4, 3, 9, 3, 4, 10, 9, 7, 7, 3, 4, 216) nas_score 297.0036926269531 params 7.90957
12/02/2022 02:06:25 - INFO - DENAS -   mutation 20/25 structure (16, 3.0, 3.0, 3.0, 3.5, 3.5, 4.0, 3.5, 3.0, 4.0, 3.0, 3.0, 3.0, 3.5

## Launch Training with Best Searched Model Structure

### Train the best searched CNN model
The input is the configuration for CNN domain best model train, edit the configuration file of `e2eaiok_denas_train_cnn.conf` for CNN best model train, and then run below commond line

In [18]:
%%bash
cd /home/vmagent/app/e2eaiok/e2eAIOK/DeNas
python -m intel_extension_for_pytorch.cpu.launch --distributed --nproc_per_node=2 --nnodes=1 train.py --domain cnn --conf /home/vmagent/app/e2eaiok/conf/denas/cv/e2eaiok_denas_train_cnn.conf

/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/runpy.py:127: RuntimeWarning: 'intel_extension_for_pytorch.cpu.launch' found in sys.modules after import of package 'intel_extension_for_pytorch.cpu', but prior to execution of 'intel_extension_for_pytorch.cpu.launch'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-12-02 02:29:57,904 - __main__ - INFO - MASTER_ADDR=127.0.0.1
2022-12-02 02:29:57,905 - __main__ - INFO - MASTER_PORT=29500
2022-12-02 02:29:57,905 - __main__ - INFO - I_MPI_PIN_DOMAIN=[0xfffff0,0xfffff0000000,]
2022-12-02 02:29:57,905 - __main__ - WARNING - Both TCMalloc and JeMalloc are not found in $CONDA_PREFIX/lib or $VIRTUAL_ENV/lib or /.local/lib/ or /usr/local/lib/ or /usr/local/lib64/ or /usr/lib or /usr/lib64 or /root/.local/lib/ so the LD_PRELOAD environment variable will not be set. This may drop the performance
2022-12-02 02:29:57,906 - __main__ - INFO - OMP_NUM_THREADS=20
2022-12-02 02:29:57,906 - __main__ - INFO 

[0] Epoch: [1]  [ 90/196]  eta: 0:00:23  lr: 0.001000  loss: 2.2986 (2.3073)  time: 0.1482  data: 0.0168
[1] Epoch: [1]  [ 90/196]  eta: 0:00:23  lr: 0.001000  loss: 2.2910 (2.3063)  time: 0.1481  data: 0.0168
[0] Epoch: [1]  [100/196]  eta: 0:00:20  lr: 0.001000  loss: 2.2932 (2.3059)  time: 0.1469  data: 0.0165
[1] Epoch: [1]  [100/196]  eta: 0:00:20  lr: 0.001000  loss: 2.2910 (2.3051)  time: 0.1469  data: 0.0178
[0] Epoch: [1]  [110/196]  eta: 0:00:17  lr: 0.001000  loss: 2.2860 (2.3039)  time: 0.1457  data: 0.0167
[1] Epoch: [1]  [110/196]  eta: 0:00:17  lr: 0.001000  loss: 2.2933 (2.3036)  time: 0.1457  data: 0.0172
[0] Epoch: [1]  [120/196]  eta: 0:00:15  lr: 0.001000  loss: 2.2868 (2.3026)  time: 0.1465  data: 0.0166
[1] Epoch: [1]  [120/196]  eta: 0:00:15  lr: 0.001000  loss: 2.2905 (2.3024)  time: 0.1465  data: 0.0163
[0] Epoch: [1]  [130/196]  eta: 0:00:12  lr: 0.001000  loss: 2.2860 (2.3009)  time: 0.1472  data: 0.0164
[1] Epoch: [1]  [130/196]  eta: 0:00:12  lr: 0.001000  

### Train the best searched ViT model
The input is the configuration for VIT domain best model train, edit the configuration file of `e2eaiok_denas_train_vit.conf` for VIT best model train, and then run below commond line

In [19]:
%%bash
cd /home/vmagent/app/e2eaiok/e2eAIOK/DeNas
python -m intel_extension_for_pytorch.cpu.launch --distributed --nproc_per_node=2 --nnodes=1 train.py --domain vit --conf /home/vmagent/app/e2eaiok/conf/denas/cv/e2eaiok_denas_train_vit.conf

/opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/runpy.py:127: RuntimeWarning: 'intel_extension_for_pytorch.cpu.launch' found in sys.modules after import of package 'intel_extension_for_pytorch.cpu', but prior to execution of 'intel_extension_for_pytorch.cpu.launch'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-12-02 02:33:35,564 - __main__ - INFO - MASTER_ADDR=127.0.0.1
2022-12-02 02:33:35,564 - __main__ - INFO - MASTER_PORT=29500
2022-12-02 02:33:35,565 - __main__ - INFO - I_MPI_PIN_DOMAIN=[0xfffff0,0xfffff0000000,]
2022-12-02 02:33:35,565 - __main__ - WARNING - Both TCMalloc and JeMalloc are not found in $CONDA_PREFIX/lib or $VIRTUAL_ENV/lib or /.local/lib/ or /usr/local/lib/ or /usr/local/lib64/ or /usr/lib or /usr/lib64 or /root/.local/lib/ so the LD_PRELOAD environment variable will not be set. This may drop the performance
2022-12-02 02:33:35,565 - __main__ - INFO - OMP_NUM_THREADS=20
2022-12-02 02:33:35,566 - __main__ - INFO 

[0] model parameters size: 26354346
[0] 12/02/2022 02:33:39 - INFO - Trainer -   model created: Vision_TransformerSuper(
[0]   (patch_embed_super): PatchembedSuper(
[0]     (proj): Conv2d(3, 640, kernel_size=(16, 16), stride=(16, 16))
[0]   )
[0]   (blocks): ModuleList(
[0]     (0): TransformerEncoderLayer(
[0]       (drop_path): Identity()
[0]       (attn): AttentionSuper(
[0]         (qkv): qkv_super(in_features=640, out_features=1920, bias=True)
[0]         (rel_pos_embed_k): RelativePosition2D_super()
[0]         (rel_pos_embed_v): RelativePosition2D_super()
[0]         (proj): LinearSuper(in_features=640, out_features=640, bias=True)
[0]         (attn_drop): Dropout(p=0.0, inplace=False)
[0]         (proj_drop): Dropout(p=0.0, inplace=False)
[0]       )
[0]       (attn_layer_norm): LayerNormSuper((640,), eps=1e-05, elementwise_affine=True)
[0]       (ffn_layer_norm): LayerNormSuper((640,), eps=1e-05, elementwise_affine=True)
[0]       (fc1): LinearSuper(in_features=640, out_featur

[1] Files already downloaded and verified
[0] Files already downloaded and verified
[1] Files already downloaded and verified
[0] Files already downloaded and verified
[1] /opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
[1]   warnings.warn(_create_warning_msg(
[1] model:Vision_TransformerSuper(
[1]   (patch_embed_super): PatchembedSuper(
[1]     (proj): Conv2d(3, 640, kernel_size=(16, 16), stride=(16, 16))
[1]   )
[1]   (blocks): ModuleList(
[1]     (0): TransformerEncoderLayer(
[1]       (drop_path): Identity()
[1]       (attn): AttentionSuper(
[1]         

[0] /opt/intel/oneapi/intelpython/latest/envs/pytorch-1.10.0/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
[0]   warnings.warn(_create_warning_msg(
[0] model:Vision_TransformerSuper(
[0]   (patch_embed_super): PatchembedSuper(
[0]     (proj): Conv2d(3, 640, kernel_size=(16, 16), stride=(16, 16))
[0]   )
[0]   (blocks): ModuleList(
[0]     (0): TransformerEncoderLayer(
[0]       (drop_path): Identity()
[0]       (attn): AttentionSuper(
[0]         (qkv): qkv_super(in_features=640, out_features=1920, bias=True)
[0]         (rel_pos_embed_k): RelativePosition2D_super()
[0]         (rel_pos_embed_v): RelativePositio

[1] /home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas/module/cv/multihead_super.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
[1]   distance_mat_v = (range_vec_k[None, :] // int(length_q ** 0.5 )  - range_vec_q[:, None] // int(length_q ** 0.5 ))
[0] /home/vmagent/app/e2eaiok/e2eAIOK/e2eAIOK/DeNas/module/cv/multihead_super.py:51: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, us